In [26]:
import pandas as pd
import numpy as np
import netCDF4 as nc
import geopandas as gpd
from shapely.geometry import box
from shapely.ops import unary_union

In [2]:
big_countries = ["AUS", "BRA", "CAN", "CHN", "IND", "RUS", "USA"]

In [9]:
emdat_damage = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_converted/emdat_droughts_converted.csv')
emdat_bcs= emdat_damage.loc[emdat_damage['ISO'].isin(big_countries)]

In [29]:
dose = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/DoseRegions.csv')
dose

,GID_0,GID_1,NAME_0,NAME_1,ENGTYPE_1,two_letter_code
0,AFG,AFG.10_1,Afghanistan,Ghor,Province,NaN
1,AFG,AFG.11_1,Afghanistan,Hilmand,Province,NaN
2,AFG,AFG.12_1,Afghanistan,Hirat,Province,NaN
3,AFG,AFG.13_1,Afghanistan,Jawzjan,Province,NaN
4,AFG,AFG.14_1,Afghanistan,Kabul,Province,NaN
...,...,...,...,...,...,...
3426,ZWE,ZWE.5_1,Zimbabwe,Mashonaland East,Province,NaN
3427,ZWE,ZWE.6_1,Zimbabwe,Mashonaland West,Province,NaN
3428,ZWE,ZWE.7_1,Zimbabwe,Masvingo,Province,NaN
3429,ZWE,ZWE.8_1,Zimbabwe,Matabeleland North,Province,NaN


In [34]:
reg_list=list(dose.loc[dose['GID_0']=='AUS', 'GID_1'])

In [80]:
emdat_bcs.loc[emdat_bcs['Start Year']>2018]

,Unnamed: 0,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,...,"Insured Damage, Adjusted ('000 US$)",damage_currentUSD_MER,damage_constant2024USD_MER,CPI,Admin Units,Entry Date,Last Update,damage_constant2015USD_PPP,damage_constant2017USD_PPP,conversion_source
612,11781,2020-9582-BRA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,3000000.0,3531938.0,84.939198,"[{""adm1_code"":685,""adm1_name"":""Rio Grande Do S...",2021-01-07,2023-09-25,5.983786e+06,6.113681e+06,GDPuc_currentUSD_to_constantXPPP
613,11782,2020-9585-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,4500000.0,5297907.0,84.939198,"[{""adm1_code"":3214,""adm1_name"":""Alabama""},{""ad...",2021-01-13,2023-09-25,4.140373e+06,4.261308e+06,GDPuc_currentUSD_to_constantXPPP
617,12163,2021-9078-CHN,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,3100000.0,3485908.0,88.929493,"[{""adm1_code"":901,""adm1_name"":""Fujian Sheng""},...",2021-02-10,2023-09-25,4.436401e+06,4.338317e+06,GDPuc_currentUSD_to_constantXPPP
627,12173,2021-9569-BRA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,3000000.0,3373459.0,88.929493,"[{""adm1_code"":680,""adm1_name"":""Parana""}]",2021-09-14,2023-09-25,5.621503e+06,5.743534e+06,GDPuc_currentUSD_to_constantXPPP
631,12177,2021-9846-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,9000000.0,10120377.0,88.929493,"[{""adm1_code"":3214,""adm1_name"":""Alabama""},{""ad...",2022-01-25,2024-04-23,7.924705e+06,8.156177e+06,GDPuc_currentUSD_to_constantXPPP
635,12541,2022-9209-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,22000000.0,22905609.0,96.046342,"[{""adm1_code"":3216,""adm1_name"":""Arizona""},{""ad...",2022-04-14,2023-09-26,1.809732e+07,1.862592e+07,GDPuc_currentUSD_to_constantXPPP
636,12542,2022-9211-BRA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,4000000.0,4164656.0,96.046342,NaN,2022-04-14,2023-09-26,6.626292e+06,6.770135e+06,GDPuc_currentUSD_to_constantXPPP
646,12552,2022-9524-CHN,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,7600000.0,7912847.0,96.046342,"[{""adm1_code"":898,""adm1_name"":""Anhui Sheng""},{...",2022-08-19,2023-09-26,1.111245e+07,1.086676e+07,GDPuc_currentUSD_to_constantXPPP
661,12920,2023-9651-BRA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,2500000.0,2500000.0,100.000000,NaN,2023-10-11,2024-01-26,4.217454e+06,4.309006e+06,GDPuc_constant2017MER_to_constantXPPP
663,12922,2023-9868-USA,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,14500000.0,14500000.0,100.000000,NaN,2024-01-17,2024-02-27,1.157280e+07,1.191083e+07,GDPuc_constant2017MER_to_constantXPPP


In [5]:
emdat_damage.columns

Index(['Unnamed: 0', 'DisNo.', 'Historic', 'Classification Key',
       'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype', 'External IDs', 'Event Name', 'ISO', 'Country',
       'Subregion', 'Region', 'Location', 'Origin', 'Associated Types',
       'OFDA/BHA Response', 'Appeal', 'Declaration',
       'AID Contribution ('000 US$)', 'Magnitude', 'Magnitude Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'damage_currentUSD_MER', 'damage_constant2024USD_MER', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update',
       'damage_constant2015USD_PPP', 'damage_constant2017USD_PPP',
       'conversion_source'

In [17]:
emdat_bcs[['Start Year','ISO',"AID Contribution ('000 US$)",'Subregion', 'Region', 'Location','Admin Units']]

,Start Year,ISO,AID Contribution ('000 US$),Subregion,Region,Location,Admin Units
31,1981,AUS,NaN,Australia and New Zealand,Oceania,"Queensland, New South Wales, Victoria States",NaN
46,1982,IND,NaN,Southern Asia,Asia,"districts of Bhiwani, Gurgaon, Hisar, Mahendar...",NaN
52,1983,CHN,NaN,Eastern Asia,Asia,Tibet,NaN
65,1983,BRA,NaN,Latin America and the Caribbean,Americas,"Northeast, Sertao",NaN
79,1984,CAN,NaN,Northern America,Americas,"Quebec, Prince Edouard Islands, Alberta, Saska...",NaN
...,...,...,...,...,...,...,...
636,2022,BRA,NaN,Latin America and the Caribbean,Americas,NaN,NaN
646,2022,CHN,NaN,Eastern Asia,Asia,"Chongqing, Hubei, Jiangxi, Tibet, Sichuan, Hun...","[{""adm1_code"":898,""adm1_name"":""Anhui Sheng""},{..."
661,2023,BRA,NaN,Latin America and the Caribbean,Americas,Amazonas state,NaN
663,2023,USA,NaN,Northern America,Americas,"Texas, Louisiana, Oklahoma, Kansas, Illinois, ...",NaN


In [23]:
country_mask= '/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/maskDoseRegionArcmin15_fractional.nc'
dataset = nc.Dataset(country_mask)
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): lon(1440), lat(720)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 MAR.27_1(lat, lon), float64 PHL.110_1(lat, lon), float64 PHL.111_1(lat, lon), float64 PHL.112_1(lat, lon), float64 PHL.113_1(lat, lon), float64 PHL.114_1(lat, lon), float64 PHL.115_1(lat, lon), float64 PHL.116_1(lat, lon), float64 MKD.100_1(lat, lon), float64 MKD.101_1(lat, lon), float64 MKD.102_1(lat, lon), float64 MKD.103_1(lat, lon), float64 MKD.104_1(lat, lon), float64 MKD.105_1(lat, lon), float64 MKD.106_1(lat, lon), float64 MKD.107_1(lat, lon), float64 BHS.98_1(lat, lon), float64 BHS.99_1(lat, lon), float64 PHL.109_1(lat, lon), float64 PHL.102_1(lat, lon), float64 PHL.101_1(lat, lon), float64 PHL.100_1(lat, lon), float64 PHL.103_1(lat, lon), float64 PHL.104_1(lat, lon), float64 PHL.105_1(lat, lon), float64 PHL.106_1(lat, lon), float64 PHL.107_1(lat, lon), float64 PHL.108_1(lat, lon),

In [36]:
area

masked_array(
  data=[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],
  mask=False,
  fill_value=1e+20)

In [92]:
shapefile_path = "/home/insauer/projects/droughts_annika/inga_data/drought_shapes_emdat.shp"

gdf = gpd.read_file(shapefile_path)

,id,country,iso3,gwno,geo_id,geolocatio,level,adm1,adm2,adm3,location,historical,hist_count,disasterty,disasterno,Shape_Leng,Shape_Area,geometry
0,05-165,China,CHN,710,40407.0,Xuwen,3,Guangdong,Zhanjiang,Xuwen,Xuwen,0.0,None,drought,1994-9096,4.431069,0.155074,"MULTIPOLYGON (((110.48153 20.39986, 110.48153 ..."
1,05-165,China,CHN,710,40411.0,Leizhou,3,Guangdong,Zhanjiang,Leizhou,Leizhou,0.0,None,drought,1994-9096,5.406979,0.313745,"MULTIPOLYGON (((110.33292 20.67014, 110.33292 ..."
50,05-165,China,CHN,710,40405.0,Chaozhou,2,Guangdong,Chaozhou,None,Chaozhou,0.0,None,drought,1994-9096,5.868990,0.283952,"MULTIPOLYGON (((116.80508 23.64617, 116.80766 ..."
51,05-165,China,CHN,710,40406.0,Meizhou,2,Guangdong,Meizhou,None,Meizhou,0.0,None,drought,1994-9096,7.355317,1.405706,"POLYGON ((115.6003 23.47092, 115.59525 23.4671..."
52,05-165,China,CHN,710,40408.0,Yangjiang,2,Guangdong,Yangjiang,None,Yangjiang,0.0,None,drought,1994-9096,9.310417,0.683736,"MULTIPOLYGON (((111.78736 21.7118, 111.78764 2..."
53,05-165,China,CHN,710,40410.0,Jieyang,2,Guangdong,Jieyang,None,Jieyang,0.0,None,drought,1994-9096,5.380475,0.460989,"MULTIPOLYGON (((116.42014 22.93681, 116.41959 ..."
54,05-165,China,CHN,710,40412.0,Shanwei,2,Guangdong,Shanwei,None,Hanwei,0.0,None,drought,1994-9096,7.132636,0.428016,"MULTIPOLYGON (((115.55224 22.79903, 115.55153 ..."
55,05-165,China,CHN,710,40413.0,Sihui,3,Guangdong,Zhaoqing,Sihui,Suixi,0.0,None,drought,1994-9096,1.811187,0.107227,"POLYGON ((112.7709 23.22562, 112.76983 23.2257..."
56,05-165,China,CHN,710,40414.0,Maoming,2,Guangdong,Maoming,None,Maoming,0.0,None,drought,1994-9096,8.337855,0.995157,"MULTIPOLYGON (((110.85152 21.50876, 110.85178 ..."


In [119]:
for ev in emdat_bcs.index:
    print(ev)
    admin_str=''
    ev_id= emdat_bcs.loc[ev, 'DisNo.'][:-4]
    iso=emdat_bcs.loc[ev, 'ISO']
    reg_list=list(dose.loc[dose['GID_0']==iso, 'GID_1'])
    for reg in reg_list:
        lat = dataset.variables['lat'][:]
        lon = dataset.variables['lon'][:]
        area = dataset.variables[reg][:]
        valid_indices = np.where(area>0)
        polygons=[]
        for i in np.arange(valid_indices[0].shape[0]):
            lon_i=valid_indices[1][i]
            lat_i=valid_indices[0][i]
            # Create a box polygon for the grid cell
            polygons.append(box(lon[lon_i] - 0.5, lat[lat_i] - 0.5, lon[lon_i] + 0.5, lat[lat_i] + 0.5))

        # Merge all polygons into a single geometry
        merged_polygons = unary_union(polygons)
        gdf_non_zero_areas = gpd.GeoDataFrame(geometry=[merged_polygons], crs='EPSG:4326')
        intersections = gdf.loc[gdf['disasterno']==ev_id].intersects(gdf_non_zero_areas.geometry.iloc[0])
        
        if intersections.sum()>0:
            admin_str=admin_str + reg + ','
    print(admin_str)
    emdat_bcs.loc[emdat_bcs['DisNo.']==ev_id+'-'+iso, 'affected_area']=admin_str

31
AUS.10_1,AUS.2_1,AUS.4_1,AUS.5_1,AUS.6_1,AUS.7_1,AUS.8_1,AUS.9_1,
46
IND.11_1,IND.12_1,IND.16_1,IND.17_1,IND.19_1,IND.25_1,IND.27_1,IND.28_1,IND.29_1,IND.2_1,IND.31_1,IND.34_1,
52
CHN.29_1,
65
BRA.21_1,BRA.24_1,
79
CAN.10_1,CAN.11_1,CAN.12_1,CAN.1_1,CAN.2_1,CAN.3_1,CAN.4_1,CAN.5_1,CAN.6_1,CAN.7_1,CAN.8_1,CAN.9_1,
80
BRA.21_1,BRA.24_1,
81
CHN.16_1,CHN.31_1,CHN.4_1,CHN.6_1,
89
IND.10_1,IND.11_1,IND.12_1,IND.15_1,IND.16_1,IND.19_1,IND.20_1,IND.25_1,IND.26_1,IND.27_1,IND.28_1,IND.29_1,IND.2_1,IND.31_1,IND.32_1,IND.34_1,IND.36_1,IND.7_1,IND.8_1,IND.9_1,
98
BRA.10_1,BRA.11_1,BRA.13_1,BRA.14_1,BRA.15_1,BRA.17_1,BRA.18_1,BRA.19_1,BRA.20_1,BRA.25_1,BRA.26_1,BRA.27_1,BRA.2_1,BRA.5_1,BRA.6_1,BRA.7_1,BRA.8_1,BRA.9_1,
110
BRA.16_1,BRA.21_1,BRA.24_1,BRA.4_1,
112
CHN.10_1,CHN.12_1,CHN.13_1,CHN.14_1,CHN.15_1,CHN.16_1,CHN.18_1,CHN.1_1,CHN.22_1,CHN.23_1,CHN.24_1,CHN.25_1,CHN.27_1,CHN.31_1,CHN.3_1,CHN.4_1,CHN.8_1,
117
CAN.11_1,CAN.13_1,CAN.1_1,CAN.2_1,CAN.3_1,CAN.6_1,CAN.9_1,
118

151
CHN.13_1,CHN.14_

In [138]:
emdat_bcs.loc[emdat_bcs['affected_area']=='', ['Start Year','ISO',"AID Contribution ('000 US$)",'Subregion', 'Region', 'Location','Admin Units','Latitude', 'Longitude', 'DisNo.'] ]

,Start Year,ISO,AID Contribution ('000 US$),Subregion,Region,Location,Admin Units,Latitude,Longitude,DisNo.
118,1988,USA,NaN,Northern America,Americas,NaN,NaN,NaN,NaN,1988-9707-USA
157,1991,AUS,NaN,Australia and New Zealand,Oceania,"Western, South",NaN,NaN,NaN,1991-9697-AUS
159,1992,CHN,NaN,Eastern Asia,Asia,North and Central,NaN,NaN,NaN,1992-9128-CHN
173,1993,RUS,NaN,Eastern Europe,Europe,NaN,NaN,NaN,NaN,1993-9521-RUS
186,1994,BRA,NaN,Latin America and the Caribbean,Americas,NaN,NaN,NaN,NaN,1994-9207-BRA
192,1995,CHN,NaN,Eastern Asia,Asia,Nord-East provinces,NaN,NaN,NaN,1995-9138-CHN
242,1999,CHN,NaN,Eastern Asia,Asia,North Region,NaN,NaN,NaN,1999-9079-CHN
386,2006,AUS,NaN,Australia and New Zealand,Oceania,NO DATA: all country has been selected,"[{""adm1_code"":468,""adm1_name"":""Australian Capi...",NaN,NaN,2006-9554-AUS
571,2017,USA,NaN,Northern America,Americas,"Upper Midwest, Northern Rockies and parts of t...","[{""adm1_code"":3216,""adm1_name"":""Arizona""},{""ad...",NaN,NaN,2017-9550-USA
588,2018,USA,NaN,Northern America,Americas,Nationwide,"[{""adm1_code"":3214,""adm1_name"":""Alabama""},{""ad...",NaN,NaN,2018-9476-USA


In [145]:
print(emdat_bcs.loc[emdat_bcs['DisNo.']=='2020-9585-USA','Admin Units'].values)

['[{"adm1_code":3214,"adm1_name":"Alabama"},{"adm1_code":3215,"adm1_name":"Alaska"},{"adm1_code":3216,"adm1_name":"Arizona"},{"adm1_code":3217,"adm1_name":"Arkansas"},{"adm1_code":3218,"adm1_name":"California"},{"adm1_code":3219,"adm1_name":"Colorado"},{"adm1_code":3225,"adm1_name":"Hawaii"},{"adm1_code":3226,"adm1_name":"Idaho"},{"adm1_code":3227,"adm1_name":"Illinois"},{"adm1_code":3228,"adm1_name":"Indiana"},{"adm1_code":3229,"adm1_name":"Iowa"},{"adm1_code":3230,"adm1_name":"Kansas"},{"adm1_code":3231,"adm1_name":"Kentucky"},{"adm1_code":3232,"adm1_name":"Louisiana"},{"adm1_code":3236,"adm1_name":"Michigan"},{"adm1_code":3237,"adm1_name":"Minnesota"},{"adm1_code":3238,"adm1_name":"Mississippi"},{"adm1_code":3239,"adm1_name":"Missouri"},{"adm1_code":3240,"adm1_name":"Montana"},{"adm1_code":3241,"adm1_name":"Nebraska"},{"adm1_code":3242,"adm1_name":"Nevada"},{"adm1_code":3245,"adm1_name":"New Mexico"},{"adm1_code":3248,"adm1_name":"North Dakota"},{"adm1_code":3249,"adm1_name":"Ohio"}

In [126]:
dis_dict={'1991-9697-AUS': 'AUS.11_1,AUS.8_1,',
          '2006-9554-AUS': 'AUS.2_1,AUS.3_1,AUS.5_1,AUS.6_1,AUS.7_1,AUS.8_1,AUS.9_1,AUS.10_1,AUS.11_1,',
          '1992-9128-CHN': 'CHN.19_1,CHN.20_1,CHN.25_1,CHN.22_1,CHN.10_1,CHN.2_1,CHN.27_1,CHN.12_1,CHN.23_1,CHN.13_1.CHN.14_1,',
          '1995-9138-CHN': 'CHN.11_1,CHN.17_1,CHN.18_1,CHN.19_1,',
          '1999-9079-CHN': 'CHN.19_1,CHN.20_1,CHN.25_1,CHN.22_1,CHN.10_1,CHN.2_1,CHN.27_1,CHN.12_1,CHN.23_1,',
          '2017-9550-USA': 'USA.3_1,USA.5_1,USA.6_1,USA.13_1,USA.16_1,USA.23_1,USA.24_1,USA.27_1,USA.29_1,USA.30_1,USA.32_1,USA.35_1,USA.38_1,USA.42_1,USA.45_1,USA.48_1,USA.50_1,USA.51_1,',
          '2018-9476-USA': 'all',
          '2020-9585-USA': 'USA.1_1,USA.2_1,USA.3_1,USA.4_1,USA.5_1,USA.6_1,USA.12_1,USA.13_1,USA.14_1,USA.15_1,USA.16_1,USA.17_1,USA.18_1,USA.19_1,USA.23_1,USA.24_1,USA.25_1,USA.26_1,USA.27_1,USA.28_1,USA.29_1,USA.32_1,USA.35_1,USA.36_1,USA.37_1,USA.38_1,USA.42_1,USA.43_1,USA.44_1,USA.45_1,USA.48_1,USA.49_1,USA.50_1,USA.51_1






         }

['1988-9707',
 '1991-9697',
 '1992-9128',
 '1993-9521',
 '1994-9207',
 '1995-9138',
 '1999-9079',
 '2006-9554',
 '2017-9550',
 '2018-9476',
 '2020-9582',
 '2020-9585',
 '2021-9078',
 '2021-9569',
 '2021-9846',
 '2022-9209',
 '2022-9211',
 '2022-9524',
 '2023-9651',
 '2023-9868',
 '2023-9872']

In [128]:
gdf.loc[gdf['disasterno'].isin(missings)]

,id,country,iso3,gwno,geo_id,geolocatio,level,adm1,adm2,adm3,location,historical,hist_count,disasterty,disasterno,Shape_Leng,Shape_Area,geometry


In [133]:
gdf.loc[gdf['disasterno']=='2006-9554']

,id,country,iso3,gwno,geo_id,geolocatio,level,adm1,adm2,adm3,location,historical,hist_count,disasterty,disasterno,Shape_Leng,Shape_Area,geometry


,id,country,iso3,gwno,geo_id,geolocatio,level,adm1,adm2,adm3,location,historical,hist_count,disasterty,disasterno,Shape_Leng,Shape_Area,geometry
0,05-165,China,CHN,710,40407.0,Xuwen,3,Guangdong,Zhanjiang,Xuwen,Xuwen,0.0,None,drought,1994-9096,4.431069,0.155074,"MULTIPOLYGON (((110.48153 20.39986, 110.48153 ..."
1,05-165,China,CHN,710,40411.0,Leizhou,3,Guangdong,Zhanjiang,Leizhou,Leizhou,0.0,None,drought,1994-9096,5.406979,0.313745,"MULTIPOLYGON (((110.33292 20.67014, 110.33292 ..."
2,05-364,Haiti,HAI,41,41100.0,Bombardopolis,3,Nord-Ouest,Mole Saint-Nicolas,Bombardopolis,Bombardopolis,0.0,None,drought,2003-9758,0.601093,0.016145,"POLYGON ((-73.30319 19.62907, -73.30375 19.629..."
3,05-366,Haiti,HAI,41,41114.0,Baraderes,3,Nippes,L'Anse-a-Veau,Baraderes,Baraderes,0.0,None,drought,2016-9052,1.621827,0.020089,"MULTIPOLYGON (((-73.73776 18.41589, -73.7407 1..."
4,05-513,Kenya,KEN,501,41967.0,Marigat,3,Baringo,Baringo South,Marigat,Marigat,0.0,None,drought,2008-9302,1.469482,0.050785,"POLYGON ((35.99149 0.33455, 35.99117 0.33456, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,05-821,South Sudan,SUD,625,45247.0,Jungoli,1,Jungoli,None,None,Jonglei,1.0,Sudan,drought,2009-9633,16.079763,10.009401,"POLYGON ((31.70858 5.85867, 31.704 5.866, 31.6..."
2934,05-821,South Sudan,SUD,625,45248.0,Upper Nile,1,Upper Nile,None,None,Upper Nile,1.0,Sudan,drought,2009-9633,14.902150,6.368242,"POLYGON ((33.18936 8.13451, 33.18888 8.13285, ..."
2935,05-821,South Sudan,SUD,625,45249.0,West Bahr-al-Ghazal,1,West Bahr-al-Ghazal,None,None,Northern Bahr Ghazal,1.0,Sudan,drought,2009-9633,15.391308,7.019659,"POLYGON ((28.49245 6.74835, 28.04549 6.74293, ..."
2936,05-844,South Sudan,SUD,625,45250.0,West Bahr-al-Ghazal,1,West Bahr-al-Ghazal,None,None,Bar El Ghazal Province,1.0,Sudan,drought,1990-9289,15.391308,7.019659,"POLYGON ((28.49245 6.74835, 28.04549 6.74293, ..."
